# Making your own `EventData`

In this example we will download data from [GWOSC](https://www.gw-openscience.org/) and make our own `EventData` (instead of loading one of the precomputed `npz` files).

In [ ]:
%matplotlib inline

In [ ]:
path_to_cogwheel = '..'

import sys
sys.path.append(path_to_cogwheel)

import subprocess
from matplotlib import pyplot as plt

import gwosc

import lal
lal.swig_redirect_standard_output_error(False)  # Or LAL may run slowly on notebooks

from cogwheel import data
from cogwheel.posterior import Posterior

In [ ]:
eventname = 'GW170817'

To download the default strain data from GWOSC, we could use the following lines and skip the next cell:

    data.download_timeseries(eventname)
    filenames = sorted((data.GWOSC_FILES_DIR/eventname).glob('*.hdf5'))
   
Here we will download [specific hdf5 data files from GWOSC](https://www.gw-openscience.org/eventapi/html/O1_O2-Preliminary/GW170817/v2) instead (with the glitch at Livingston cleaned):

In [ ]:
urls = gwosc.locate.get_event_urls('GW170817', version=2)  # Cleaned GW170817

outdir = data.GWOSC_FILES_DIR/eventname
subprocess.run(['wget', '-P', outdir, *urls])

filenames = [outdir/url.split('/')[-1] for url in urls]

In [ ]:
detector_names = ''.join(filename.name[0] for filename in filenames)
tgps = gwosc.datasets.event_gps(eventname)

Make `EventData` making sure we keep a length of data enough to contain the signal, and that the Nyquist frequency is high enough to capture all available SNR.

This estimates the noise power spectrum with the Welch method from the full file and crops, high-passes and whitens a chunk of data. The duration of the chunk of data, that of the Welch segments and the Nyquist frequency are determined by the arguments to `from_timeseries`.

In [ ]:
event_data = data.EventData.from_timeseries(
    filenames, eventname.split('-')[0], detector_names, tgps, t_before=128., fmax=1600.)

Plot whitening filter (inverse of the noise amplitude spectral density times a high-pass)

In [ ]:
plt.figure()
plt.plot(event_data.frequencies, event_data.wht_filter.T,
         label=list(event_data.detector_names))
plt.legend()

plt.xlabel('Frequency (Hz)')
plt.ylabel(r'Whitening filter ($\sqrt{\rm Hz}$)')
plt.xlim(0)
plt.ylim(0);

Plot spectrogram (full file and zoom-in)

In [ ]:
event_data.specgram()

In [ ]:
event_data.specgram((-1.5, .5), nfft=200)

We can use this `EventData` to make a `Posterior` object.

*Note:* `mchirp_guess` is detector frame chirp-mass, should be less than a few sigmas away from the truth. This becomes important for low mass systems like GW170817 whose chirp mass is very well measured.

In [ ]:
event_data.to_npz()

In [ ]:
mchirp_guess = 1.198
post = Posterior.from_event(event_data, mchirp_guess, 'IMRPhenomXPHM', 'LVCPrior')

We can save the event data and/or the posterior to use later:
    
    event_data.to_npz()
    post.to_json(dirname)  # Then load with cogwheel.utils.read_json()